In [16]:
from google.cloud import bigquery
import pandas as pd
from google.oauth2.service_account import Credentials


In [17]:
credentials = Credentials.from_service_account_file('config/gcp_credentials.json')

In [18]:
data = pd.read_csv('source/candidates.csv', sep=';')
#print(data)
print(data.head())
print(data.dtypes)


   First Name   Last Name                      Email Application Date  \
0  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
1      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
2       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
3        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
4       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

   Country  YOE  Seniority                         Technology  \
0   Norway    2     Intern                      Data Engineer   
1   Panama   10     Intern                      Data Engineer   
2  Belarus    4  Mid-Level                     Client Success   
3  Eritrea   25    Trainee                          QA Manual   
4  Myanmar   13  Mid-Level  Social Media Community Management   

   Code Challenge Score  Technical Interview Score  
0                     3                          3  
1                     2                         10  
2          

In [19]:
# clean and format the column names
data.columns = [col.strip().lower().replace(' ', '_') for col in data.columns]
print(data.columns)

Index(['first_name', 'last_name', 'email', 'application_date', 'country',
       'yoe', 'seniority', 'technology', 'code_challenge_score',
       'technical_interview_score'],
      dtype='object')


In [20]:
data_sorted = data.sort_values(by='email', ascending=False).head(100)
data_sorted.to_csv('out/sorted_data.csv', index=False, header=True)

In [28]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   first_name                 50000 non-null  object
 1   last_name                  50000 non-null  object
 2   email                      50000 non-null  object
 3   application_date           50000 non-null  object
 4   country                    50000 non-null  object
 5   yoe                        50000 non-null  int64 
 6   seniority                  50000 non-null  object
 7   technology                 50000 non-null  object
 8   code_challenge_score       50000 non-null  int64 
 9   technical_interview_score  50000 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 3.8+ MB
None


In [22]:
# Set up the BigQuery client
client = bigquery.Client.from_service_account_json('config/gcp_credentials.json')

In [23]:
# Create a new dataset
dataset_id = 'data_test'
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
try:
    dataset = client.create_dataset(dataset)
except Exception:
    pass

In [24]:
table_id = 'candidates'
schema = [
    bigquery.SchemaField('first_name', 'STRING'),
    bigquery.SchemaField('last_name', 'STRING'),
    bigquery.SchemaField('email', 'STRING'),
    bigquery.SchemaField('application_date', 'DATE'),
    bigquery.SchemaField('country', 'STRING'),
    bigquery.SchemaField('yoe', 'INTEGER'),
    bigquery.SchemaField('seniority', 'STRING'),
    bigquery.SchemaField('technology', 'STRING'),
    bigquery.SchemaField('code_challenge_score', 'INTEGER'),
    bigquery.SchemaField('technical_interview_score', 'INTEGER')
]

In [25]:
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref, schema=schema)


In [26]:
# # Load the data into the table
# job_config = bigquery.LoadJobConfig(schema=schema)
# job_config.autodetect = True
# job_config.write_disposition = 'WRITE_TRUNCATE'
# job_config.source_format = bigquery.SourceFormat.CSV
# job = client.load_table_from_dataframe(data, table, job_config=job_config)
# job.result()

In [27]:
data.to_gbq(destination_table='data_test.candidates', 
            project_id='tech-tests-381614',
            if_exists='replace',
            credentials=credentials)